<a href="https://colab.research.google.com/github/k-washi/3d-Bonet-Boost/blob/master/notebook/bonet_qbs_simple_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3D - BoNetのスタート

樹木データへの適用 (ただし、テスト用に作成したデータセット使用)



# 3D-BoNetのインストール
[Learning Object Bounding Boxes for 3D Instance Segmentation on Point Clouds](https://arxiv.org/abs/1906.01140)

https://github.com/Yang7879/3D-BoNet
---



In [0]:
!rm -rf 3D-BoNet

In [8]:
!git clone https://github.com/k-washi/3D-BoNet.git

Cloning into '3D-BoNet'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 441 (delta 15), reused 26 (delta 10), pack-reused 409
Receiving objects: 100% (441/441), 110.54 MiB | 33.85 MiB/s, done.
Resolving deltas: 100% (249/249), done.


#python ライブラリの確認

tensorflow 1.15

In [0]:
%tensorflow_version 1.x
!pip show tensorflow
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
device_name

Name: tensorflow
Version: 1.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: google-pasta, tensorflow-estimator, numpy, six, wrapt, opt-einsum, termcolor, wheel, keras-applications, tensorboard, astor, grpcio, gast, keras-preprocessing, protobuf, absl-py
Required-by: stable-baselines, magenta, fancyimpute


'/device:GPU:0'

# データセットのダウンロード

Google Driveにデータセットを置くと仮定

/content/drive/My Drive/data/Data_S3DIS.zip

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


データを解凍

In [0]:
!apt-get -y install p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
!mkdir qbs_train_data
!cp /content/drive/My\ Drive/data/qbs_test_data/* -d qbs_train_data

# 3D-BoNet: Compile tensorflow operation by cuda 

In [0]:
!nvcc --version
!which nvcc

!ls -l /usr/lib/x86_64-linux-gnu/libcudnn.so*
!ls /usr/lib64-nvidia/libcuda*

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
/usr/local/cuda/bin/nvcc
lrwxrwxrwx 1 root root        29 Dec  5 22:54 /usr/lib/x86_64-linux-gnu/libcudnn.so -> /etc/alternatives/libcudnn_so
lrwxrwxrwx 1 root root        17 Oct 27 21:49 /usr/lib/x86_64-linux-gnu/libcudnn.so.7 -> libcudnn.so.7.6.5
-rw-r--r-- 1 root root 428711256 Oct 27 21:49 /usr/lib/x86_64-linux-gnu/libcudnn.so.7.6.5
/usr/lib64-nvidia/libcuda.so	/usr/lib64-nvidia/libcuda.so.418.67
/usr/lib64-nvidia/libcuda.so.1


In [0]:
!ls /usr/local/cuda-10.1/

bin	extras	 libnsight  nsightee_plugins  README   src	version.txt
compat	include  libnvvp    nvml	      samples  targets
doc	lib64	 LICENSE    nvvm	      share    tools


compileに必要なcudaディレクトリとGC上で実行することを引数で与える

In [0]:
!python 3D-BoNet/tf_ops/cu_compile_v1.15.py --cuda /usr/local/cuda-10.1 --googleColab

------------------------------
 
!/usr/local/cuda-10.1/bin/nvcc 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_g.cu -o 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_g.cu.o -c -O2 -DGOOGLE_CUDA=1 -x cu -Xcompiler -fPIC
!/usr/local/cuda-10.1/bin/nvcc 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_g.cu -o 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_g.cu.o -c -O2 -DGOOGLE_CUDA=1 -x cu -Xcompiler -fPIC
!g++ -std=c++11 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_g.cu.o -o 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_so.so -shared -fPIC -I/tensorflow-1.15.0/python3.6/tensorflow_core/include -D_GLIBCXX_USE_CXX11_ABI=0 -I /usr/local/cuda-10.1/include -I/tensorflow-1.15.0/python3.6/tensorflow_core/include/external/nsync/public -L/tensorflow-1.15.0/python3.6/tensorflow_core -l:libtensorflow_framework.so.1 -lcudart -L /usr/local/cuda-10.1/lib64 -O2
!g++ -std=c++11 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling.cpp 3D-BoNet/tf_ops/../tf_op

上のpythonの出力コマンドをコピーして実行 (2020/3/9時点では、以下のコマンド)

In [9]:
!/usr/local/cuda-10.1/bin/nvcc 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_g.cu -o 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_g.cu.o -c -O2 -DGOOGLE_CUDA=1 -x cu -Xcompiler -fPIC
!/usr/local/cuda-10.1/bin/nvcc 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_g.cu -o 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_g.cu.o -c -O2 -DGOOGLE_CUDA=1 -x cu -Xcompiler -fPIC
!g++ -std=c++11 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_g.cu.o -o 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping_so.so -shared -fPIC -I/tensorflow-1.15.0/python3.6/tensorflow_core/include -D_GLIBCXX_USE_CXX11_ABI=0 -I /usr/local/cuda-10.1/include -I/tensorflow-1.15.0/python3.6/tensorflow_core/include/external/nsync/public -L/tensorflow-1.15.0/python3.6/tensorflow_core -l:libtensorflow_framework.so.1 -lcudart -L /usr/local/cuda-10.1/lib64 -O2
!g++ -std=c++11 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling.cpp 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_g.cu.o -o 3D-BoNet/tf_ops/../tf_ops/sampling/tf_sampling_so.so -shared -fPIC -I/tensorflow-1.15.0/python3.6/tensorflow_core/include -D_GLIBCXX_USE_CXX11_ABI=0 -I /usr/local/cuda-10.1/include -I/tensorflow-1.15.0/python3.6/tensorflow_core/include/external/nsync/public -L/tensorflow-1.15.0/python3.6/tensorflow_core -l:libtensorflow_framework.so.1 -lcudart -L /usr/local/cuda-10.1/lib64 -O2
!g++ -std=c++11 3D-BoNet/tf_ops/../tf_ops/interpolation/tf_interpolate.cpp -o 3D-BoNet/tf_ops/../tf_ops/interpolation/tf_interpolate_so.so -shared -fPIC -I/tensorflow-1.15.0/python3.6/tensorflow_core/include -D_GLIBCXX_USE_CXX11_ABI=0 -I /usr/local/cuda-10.1/include -I/tensorflow-1.15.0/python3.6/tensorflow_core/include/external/nsync/public -L/tensorflow-1.15.0/python3.6/tensorflow_core -l:libtensorflow_framework.so.1 -lcudart -L /usr/local/cuda-10.1/lib64 -O2

3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp: In lambda function:
3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp:22:44: warning: ignoring return value of ‘tensorflow::Status tensorflow::shape_inference::InferenceContext::WithRank(tensorflow::shape_inference::ShapeHandle, tensorflow::int64, tensorflow::shape_inference::ShapeHandle*)’, declared with attribute warn_unused_result [-Wunused-result]
         c->WithRank(c->input(1), 3, &dims2);
                                            ^
In file included from 3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp:8:0:
/tensorflow-1.15.0/python3.6/tensorflow_core/include/tensorflow/core/framework/shape_inference.h:394:10: note: declared here
   Status WithRank(ShapeHandle shape, int64 rank,
          ^~~~~~~~
3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp: In lambda function:
3D-BoNet/tf_ops/../tf_ops/grouping/tf_grouping.cpp:47:44: warning: ignoring return value of ‘tensorflow::Status tensorflow::shape_inference::InferenceContext

In [10]:
!ls 3D-BoNet/tf_ops/sampling/

__init__.py		tf_sampling.cpp   tf_sampling_g.cu.o  tf_sampling_so.so
tf_sampling_compile.sh	tf_sampling_g.cu  tf_sampling.py


# 3D-BoNet の学習を実行する

In [11]:
!python 3D-BoNet/qbs_train.py -i ./qbs_train_data -s 0 -e 1 -c ./3D-BoNet/config/config.ini

re_train: False
./log/test_sum/ : deleted and then created!
./log/train_sum/ : deleted and then created!
./log/train_mod/ : deleted and then created!


/content
2020-03-19 04:47:57.426578: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow w

In [0]:
%tensorflow_version 1.x
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [0]:
!mkdir elog
!mv log/* elog

mkdir: cannot create directory ‘elog’: File exists
mv: cannot move 'log/test_sum' to 'elog/test_sum': Directory not empty
mv: cannot move 'log/train_mod' to 'elog/train_mod': Directory not empty
mv: cannot move 'log/train_sum' to 'elog/train_sum': Directory not empty


In [12]:
!python 3D-BoNet/qbs_eval.py -i ./qbs_train_data -m ./log/train_mod/model_fin.cptk -c ./3D-BoNet/config/config.ini



/content
2020-03-19 04:49:44.829095: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2020-03-19 04:49:46.510201: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-03-19 04:49:46.510412: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x20daf4